In [1]:
import pandas as pd
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

In [2]:
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')

In [3]:
audio_features.head()

,acousticness,danceability,energy,id,instrumentalness,key,liveness,loudness,mode,sp_id,speechiness,tempo,time_signature,track_uri,valence
0,0.02040,0.708,0.610,1,0.000004,2,0.0641,-4.522,1,0XLOf9LhyazPX9Ld8jPiUq,0.0574,123.038,4,spotify:track:0XLOf9LhyazPX9Ld8jPiUq,0.305
1,0.18300,0.593,0.868,2,0.000000,1,0.3650,-3.573,0,7aF09WaavZAmAWuUeYxlYD,0.2950,104.988,4,spotify:track:7aF09WaavZAmAWuUeYxlYD,0.496
2,0.00683,0.600,0.810,3,0.210000,0,0.1550,-4.749,1,0tKcYR2II1VCQWT79i5NrW,0.0479,167.880,4,spotify:track:0tKcYR2II1VCQWT79i5NrW,0.298
3,0.01620,0.503,0.918,4,0.024400,11,0.1860,-5.010,1,6TWSVHx6z6E42JiwloGv1k,0.0464,140.000,4,spotify:track:6TWSVHx6z6E42JiwloGv1k,0.418
4,0.01850,0.760,0.561,5,0.000019,1,0.1990,-8.697,1,5QqyRUZeBE04yJxsD1OC0I,0.1340,134.983,4,spotify:track:5QqyRUZeBE04yJxsD1OC0I,0.121


In [4]:
audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2262133 entries, 0 to 2262132
Data columns (total 15 columns):
acousticness        float64
danceability        float64
energy              float64
id                  int64
instrumentalness    float64
key                 object
liveness            float64
loudness            float64
mode                object
sp_id               object
speechiness         float64
tempo               float64
time_signature      object
track_uri           object
valence             float64
dtypes: float64(9), int64(1), object(5)
memory usage: 258.9+ MB


In [5]:
audio_features['key'] = audio_features['key'].astype(int)
audio_features['mode'] = audio_features['mode'].astype(int)
audio_features['time_signature'] = audio_features['time_signature'].astype(int)

#### 418 Songs Have Incorrect Audio Features
Removing them from possible recommendations

In [6]:
track_audio_features = audio_features[audio_features.energy == 0]

In [7]:
len(track_audio_features)

418

In [8]:
audio_features = audio_features[audio_features.energy != 0]

#### Dropping sp_id and id - Irrelevant

In [9]:
audio_features = audio_features.drop(['sp_id','id'],axis=1)

#### Filling Null Valence with Mean

In [10]:
audio_features.isnull().sum()

acousticness        0
danceability        0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
track_uri           0
valence             1
dtype: int64

In [11]:
audio_features['valence'] = audio_features['valence'].fillna(audio_features['valence'].mean())

In [12]:
audio_features.isnull().sum()

acousticness        0
danceability        0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
track_uri           0
valence             0
dtype: int64

#### Lots of songs have 0 in some features. 
This is mainly attributed to them being very short in length (less 20 seconds).<br>
Will remove for recommendations as it could cause problems.<br>
Other 0 values are correct after sampling random songs.

In [13]:
audio_features[audio_features['tempo'] == 0].head(5)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_uri,valence
6698,0.0321,0.0,0.331000,0.917,0,0.0831,-8.115,1,0.0,0.0,0,spotify:track:4DG4um6R0wx4WrqhhACc5g,0.0
9371,0.0455,0.0,0.639000,0.000,2,0.6630,-6.115,0,0.0,0.0,0,spotify:track:2iq9JouHeGBsOT0XJwSlz2,0.0
9373,0.9770,0.0,0.104000,0.000,7,0.1190,-9.308,1,0.0,0.0,0,spotify:track:3mviGvUGUp1Pii6qm5uAdv,0.0
9377,0.0000,0.0,0.000379,0.000,2,0.0000,-23.264,1,0.0,0.0,0,spotify:track:2LZK6VvqPqkoi1Ujvef03z,0.0
11472,0.6180,0.0,0.072100,0.694,2,0.0000,-22.480,1,0.0,0.0,0,spotify:track:4Fz1WWr5o0OrlIcZxcyZtK,0.0


#### Averages

In [14]:
print('acousticness mean:',audio_features['acousticness'].mean())
print('danceability mean:',audio_features['danceability'].mean())
print('energy mean:',audio_features['energy'].mean())
print('instrumentalness mean:',audio_features['instrumentalness'].mean())
print('liveness mean:',audio_features['liveness'].mean())
print('loudness mean:',audio_features['loudness'].mean())
print('speechiness mean:',audio_features['speechiness'].mean())
print('tempo mean:',audio_features['tempo'].mean())
print('valence mean:',audio_features['valence'].mean())
print('key mean:',audio_features['key'].mean())

acousticness mean: 0.3550146365193845
danceability mean: 0.5505726613653797
energy mean: 0.5850353599919527
instrumentalness mean: 0.22083376861014764
liveness mean: 0.2093197049274555
loudness mean: -9.654175217036629
speechiness mean: 0.08939162312669803
tempo mean: 120.00268459465491
valence mean: 0.47591965383425994
key mean: 5.262110389682166


In [15]:
print('acousticness max:',audio_features['acousticness'].max())
print('danceability max:',audio_features['danceability'].max())
print('energy max:',audio_features['energy'].max())
print('instrumentalness max:',audio_features['instrumentalness'].max())
print('liveness max:',audio_features['liveness'].max())
print('loudness max:',audio_features['loudness'].max())
print('speechiness max:',audio_features['speechiness'].max())
print('tempo max:',audio_features['tempo'].max())
print('valence max:',audio_features['valence'].max())
print('key max:',audio_features['key'].max())
print('\n')
print('acousticness min:',audio_features['acousticness'].min())
print('danceability min:',audio_features['danceability'].min())
print('energy min:',audio_features['energy'].min())
print('instrumentalness min:',audio_features['instrumentalness'].min())
print('liveness min:',audio_features['liveness'].min())
print('loudness min:',audio_features['loudness'].min())
print('speechiness min:',audio_features['speechiness'].min())
print('tempo min:',audio_features['tempo'].min())
print('valence min:',audio_features['valence'].min())
print('key min:',audio_features['key'].min())

acousticness max: 0.996
danceability max: 0.996
energy max: 1.0
instrumentalness max: 1.0
liveness max: 1.0
loudness max: 4.923
speechiness max: 0.971
tempo max: 249.987
valence max: 1.0
key max: 11


acousticness min: 0.0
danceability min: 0.0
energy min: 1.32e-05
instrumentalness min: 0.0
liveness min: 0.0
loudness min: -60.0
speechiness min: 0.0
tempo min: 0.0
valence min: 0.0
key min: 0


#### Overwriting audio_features with new count column

In [17]:
audio_features.to_pickle('../Pickle_Files/audio_features.pkl')